In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm 
import time

In [2]:
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D,Conv3D,Conv2DTranspose,MaxPooling3D,UpSampling3D
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.layers import concatenate 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

Try model with and without loss function

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    data_model = '/content/gdrive/MyDrive/Zeta_Surgical_Segmentation/data_model/'
except:
    pass

Mounted at /content/gdrive


In [4]:
data_model = '/content/gdrive/MyDrive/Zeta_Surgical_Segmentation/data_model/'
X_train = np.load(data_model + 'X_train.npy')
Y_train = np.load(data_model + 'Y_train.npy')
X_test = np.load(data_model + 'X_test.npy')
Y_test = np.load(data_model + 'Y_test.npy')

In [11]:
#Weighted binary cross entropy 

def create_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        
        # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce
        
        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
input_size = (80,128,128,1)


inputs = Input(input_size)
conv1 = Conv3D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv3D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
conv2 = Conv3D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv3D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
conv3 = Conv3D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv3D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
conv4 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)

conv5 = Conv3D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv3D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv3D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 4)
conv6 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv3D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv3D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 4)
conv7 = Conv3D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv3D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv3D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 4)
conv8 = Conv3D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv3D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv3D(8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling3D(size = (2,2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 4)
conv9 = Conv3D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv3D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv3D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv3D(1, 1, activation = 'sigmoid')(conv9)



In [12]:
model = Model(inputs = inputs, outputs = conv10)

#For trial with weighted binary cross entropy 
#Assigning lower weights to class 0 and higher to class 1 
model_loss = create_weighted_binary_crossentropy(zero_weight = 0.2, one_weight = 0.8 )

#Learning rate= 0.0002
model.compile(optimizer = Adam(learning_rate = 2e-4), loss = model_loss , metrics = ['accuracy', f1_m])

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 128, 12  0           []                               
                                8, 1)]                                                            
                                                                                                  
 conv3d (Conv3D)                (None, 80, 128, 128  224         ['input_1[0][0]']                
                                , 8)                                                              
                                                                                                  
 conv3d_1 (Conv3D)              (None, 80, 128, 128  1736        ['conv3d[0][0]']                 
                                , 8)                                                        

In [13]:
#Creating checkpoints to save model after each epoch 
#Saving model if loss is reduced from previous epoch
epochs = 10
filepath = data_model + "model_{epoch:03d}-{loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')
callbacks_list = [checkpoint]

In [15]:
history = model.fit(X_train, 
                    Y_train, 
                    epochs=epochs, 
                    batch_size=64,
                    verbose=1,
                    validation_data=(X_test, Y_test), callbacks=[checkpoint])

Epoch 1/10


ResourceExhaustedError: ignored